In [1]:
from Game import *

Version:  0.2.23


# Nim

Rules:

1. start with 21 sticks -- state = number of sticks
2. turns alternate taking 1,2, or 3 sticks
3. player taking last stick loses

In [7]:
def initial_state():
    return 25

def show_state(state):
    print("There are ",state,"sticks.")
    
def valid_moves(state,player):
    if state==1:
        return [1]
    elif state==2:
        return [1,2]
    else:
        return [1,2,3]
    
def update_state(state,player,move):
    # move = number of sticks to pick up
    new_state=state-move # remove the sticks
    return new_state

def win_status(state,player):
    if state==0:
        return 'lose'
    elif state==1:
        return 'win'
    else:
        return None
    
    # there is no stalemate

In [8]:
def human_move(state,player):
    print("Player ",player)
    move=int(input("How many sticks?"))
    return move

human_agent=Agent(human_move)

In [9]:
def random_move(state,player):
    possible_moves=valid_moves(state,player)
    move=random.choice(possible_moves)
    return move
random_agent=Agent(random_move)

In [10]:
from Game.minimax import *
def minimax_move(state,player):

    values,moves=minimax_values(state,player,display=False)
    return top_choice(moves,values)


minimax_agent=Agent(minimax_move)

In [11]:
def skittles_move(state,player,info):
    S=info.S
    last_action=info.last_action
    last_state=info.last_state
    
    
    # if Ive never seen this state before
    if not state in S:
        actions=valid_moves(state,player)

        S[state]=Table()
        for action in actions:
            S[state][action]=3     
    
    move=weighted_choice(S[state])  # weighted across actions
    
    # what if there are no skittles for a particular state?
    # move is None in that case
    
    if move is None:
        # learn a little bit
        if last_state:
            S[last_state][last_action]=S[last_state][last_action]-1
            if S[last_state][last_action]<0:
                S[last_state][last_action]=0
        
        move=random_move(state,player)
    
    return move

def skittles_after(status,player,info):
    S=info.S
    last_action=info.last_action
    last_state=info.last_state

    if status=='lose':
        # learn a little bit
        S[last_state][last_action]=S[last_state][last_action]-1
        if S[last_state][last_action]<0:
            S[last_state][last_action]=0
        
    


skittles_agent=Agent(skittles_move)
skittles_agent.S=Table()
skittles_agent.post=skittles_after


Some miscellaneous table stuff

In [9]:
S=Table()

In [16]:
state=Board(3,3)
state[4]=1
state

 0  0  0 
 0  1  0 
 0  0  0 

In [17]:
state

 0  0  0 
 0  1  0 
 0  0  0 

In [18]:
S[state]=2

In [19]:
S

{21: 4, (0, 0, 0, 0, 1, 0, 0, 0, 0): 2}

In [24]:
S=Table()
state=initial_state()
actions=valid_moves(state,1)

S[state]=Table()
for action in actions:
    S[state][action]=3  # nunber of skittles

In [25]:
S

{21: {1: 3, 2: 3, 3: 3}}

In [26]:
state=2
S[state]=Table()
actions=valid_moves(state,1)
for action in actions:
    S[state][action]=3  # nunber of skittles

In [27]:
S

{21: {1: 3, 2: 3, 3: 3}, 2: {1: 3, 2: 3}}

In [52]:
skittles_agent.S=Table()

In [26]:
g=Game(1000)
g.display=False
g.run(random_agent,skittles_agent);

In [27]:
g.report()

Total number of games:  1000
Winning 0.00 percent
Losing 100.00 percent
Tie 0.00 percent


In [55]:
skittles_agent.S

{18: {1: 3, 2: 0, 3: 0},
 13: {1: 0, 2: 0, 3: 0},
 9: {1: 0, 2: 0, 3: 0},
 5: {1: 0, 2: 0, 3: 0},
 19: {1: 0, 2: 3, 3: 0},
 17: {1: 0, 2: 0, 3: 0},
 16: {1: 0, 2: 0, 3: 3},
 20: {1: 0, 2: 0, 3: 3},
 11: {1: 0, 2: 3, 3: 2},
 15: {1: 0, 2: 3, 3: 0},
 14: {1: 3, 2: 0, 3: 1},
 12: {1: 2, 2: 0, 3: 3},
 10: {1: 3, 2: 2, 3: 0},
 8: {1: 0, 2: 2, 3: 3},
 7: {1: 2, 2: 3, 3: 2},
 3: {1: 0, 2: 3, 3: 2},
 6: {1: 3, 2: 1, 3: 1},
 4: {1: 3, 2: 3, 3: 3},
 2: {1: 3, 2: 3}}

In [33]:
SaveTable(skittles_agent.S,'nim skittles table.json')

In [32]:
sorted(list(skittles_agent.S.keys()))

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [57]:
skittles_agent.S=LoadTable('nim skittles table.json')

In [6]:
g=Game()
g.run(minimax_agent,random_agent)

====
Game  1
There are  21 sticks.
  Choice Time: 0.15351605415344238 seconds 
Player 1 moves 1
There are  20 sticks.
Player 2 moves 2
There are  18 sticks.
  Choice Time: 0.0407710075378418 seconds 
Player 1 moves 1
There are  17 sticks.
Player 2 moves 3
There are  14 sticks.
  Choice Time: 0.005802154541015625 seconds 
Player 1 moves 1
There are  13 sticks.
Player 2 moves 1
There are  12 sticks.
  Choice Time: 0.0020380020141601562 seconds 
Player 1 moves 3
There are  9 sticks.
Player 2 moves 3
There are  6 sticks.
  Choice Time: 8.797645568847656e-05 seconds 
Player 1 moves 1
There are  5 sticks.
Player 2 moves 1
There are  4 sticks.
  Choice Time: 2.8133392333984375e-05 seconds 
Player 1 moves 3
There are  1 sticks.
Player  1 won.


[1]

In [7]:
g=Game(10)
g.display=False
g.run(minimax_agent,random_agent)

  Choice Time: 0.15744709968566895 seconds 
  Choice Time: 0.017589807510375977 seconds 
  Choice Time: 0.0011348724365234375 seconds 
  Choice Time: 0.00014209747314453125 seconds 
  Choice Time: 1.5974044799804688e-05 seconds 
  Choice Time: 0.15554213523864746 seconds 
  Choice Time: 0.02327895164489746 seconds 
  Choice Time: 0.004106998443603516 seconds 
  Choice Time: 0.0007109642028808594 seconds 
  Choice Time: 5.316734313964844e-05 seconds 
  Choice Time: 1.0013580322265625e-05 seconds 
  Choice Time: 0.160170316696167 seconds 
  Choice Time: 0.0248870849609375 seconds 
  Choice Time: 0.0032148361206054688 seconds 
  Choice Time: 0.0007231235504150391 seconds 
  Choice Time: 0.0001380443572998047 seconds 
  Choice Time: 9.059906005859375e-06 seconds 
  Choice Time: 0.15797185897827148 seconds 
  Choice Time: 0.014777898788452148 seconds 
  Choice Time: 0.0014598369598388672 seconds 
  Choice Time: 0.00029015541076660156 seconds 
  Choice Time: 1.5020370483398438e-05 seconds 
 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]